# **Load the data**

In [ ]:
#####################################
# Replace with your path of dataset #
##################################### 
dataset_path = r"C:\Users\Badr\Desktop\mtcaic3"

# **Preprocessing**

In [4]:
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import FastICA
import matplotlib.pyplot as plt
from scipy.signal import iirnotch, filtfilt, butter , coherence, csd, welch, hilbert
import scipy.signal as signal
from scipy.stats import kurtosis , skew

In [ ]:


path_of_MI_diret_train = os.path.join(dataset_path, "MI", "train")
path_of_MI_diret_test = os.path.join(dataset_path, "MI", "test")
path_of_MI_diret_valid = os.path.join(dataset_path, "MI", "validation")

subjects_of_train , subjects_of_test , subjects_of_valid = os.listdir(path_of_MI_diret_train) , os.listdir(path_of_MI_diret_test) , os.listdir(path_of_MI_diret_valid)
print(f"train subjects : {subjects_of_train}")
print(f"validation subjects : {subjects_of_valid}")
print(f"test subjects : {subjects_of_test}")
train_dataset , test_dataset , valid_dataset = [] , [] , []

train subjects : ['S1', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S2', 'S20', 'S21', 'S22', 'S23', 'S24', 'S25', 'S26', 'S27', 'S28', 'S29', 'S3', 'S30', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9']
validation subjects : ['S31', 'S32', 'S33', 'S34', 'S35']
test subjects : ['S36', 'S37', 'S38', 'S39', 'S40']


In [11]:
# ---------------------------------------------------------------------------
# Independent Component Analysis (ICA) helper
# ---------------------------------------------------------------------------
def apply_ica(data):
    data = np.asarray(data)                           # ensure ndarray
    if data.ndim == 1:                                # reshape 1‑D → 2‑D
        data = data[:, None]

    ica = FastICA(                                    # ICA model
        n_components=data.shape[1],
        random_state=42,
        max_iter=2000,
        tol=1e-5
    )

    S = ica.fit_transform(data)                       # sources
    k = np.abs(kurtosis(S, axis=0, fisher=False))     # kurtosis per source
    S[:, k > 5] = 0                                   # suppress artifactual comps
    cleaned = ica.inverse_transform(S)                # back‑project
    return cleaned.squeeze()                          # drop singleton dims


# ---------------------------------------------------------------------------
# End‑to‑end preprocessing + feature extraction
# ---------------------------------------------------------------------------
def preprocessing_and_loading(
    subjects_of_train,
    path_of_MI_diret_train,
    output_csv="MI_features.csv"
):
    """Pipeline: load raw MI data → preprocess → extract features → CSV.

    Each row in the resulting CSV corresponds to one trial.
    """
    # ----------------------------- constants --------------------------------
    fs = 250                          # sampling frequency (Hz)
    baseline_end = fs                 # 1‑s baseline (first 250 samples)
    fbcsp_bands = [                   # FBCSP sub‑bands (Hz)
        (8, 12), (12, 16), (16, 20),
        (20, 24), (24, 28), (28, 32)
    ]

    # -------------------- helper: band‑power (FFT) --------------------------
    def bandpower(sig, f_lo, f_hi):
        """Relative power within [f_lo, f_hi] using periodogram."""
        freqs = np.fft.rfftfreq(len(sig), 1 / fs)
        pxx = np.abs(np.fft.rfft(sig)) ** 2
        band_mask = (freqs >= f_lo) & (freqs <= f_hi)
        return np.sum(pxx[band_mask]) / len(sig)

    # ----------------------------- main loop --------------------------------
    rows, X_all = [], []

    for sbj in subjects_of_train:
        # Notch + band‑pass filters
        f0, Q = 50, 30                               # notch (50 Hz line noise)
        b_notch, a_notch = iirnotch(f0, Q, fs)
        b_bp, a_bp = butter(4, [8 / (fs / 2), 30 / (fs / 2)], btype="band")

        sbj_path = os.path.join(path_of_MI_diret_train, sbj)

        for sess in os.listdir(sbj_path):
            # Assumes a single CSV per session directory
            csv_path = os.path.join(sbj_path, sess,
                                    os.listdir(os.path.join(sbj_path, sess))[0])
            data_df = pd.read_csv(csv_path)

            for tr in range(1, 11):                  # 10 trials per session
                # ------------------------------------------------------------------
                # 1) Segment & ICA cleaning
                # ------------------------------------------------------------------
                seg = data_df.iloc[2250 * (tr - 1):2250 * tr, 1:9].to_numpy()
                seg = apply_ica(seg)

                # ------------------------------------------------------------------
                # 2) Filtering (notch → band‑pass)
                # ------------------------------------------------------------------
                c3 = filtfilt(b_notch, a_notch, seg[:, 1] - seg[:, 1].mean())
                cz = filtfilt(b_notch, a_notch, seg[:, 2] - seg[:, 2].mean())
                c4 = filtfilt(b_notch, a_notch, seg[:, 3] - seg[:, 3].mean())

                for ch in range(seg.shape[1]):                       # band‑pass all
                    seg[:, ch] = filtfilt(b_bp, a_bp, seg[:, ch])

                c3 = filtfilt(b_bp, a_bp, c3)
                cz = filtfilt(b_bp, a_bp, cz)
                c4 = filtfilt(b_bp, a_bp, c4)

                # ------------------------------------------------------------------
                # 3) Baseline correction (ERD/ERS)
                # ------------------------------------------------------------------
                c3 -= c3[:baseline_end].mean()
                cz -= cz[:baseline_end].mean()
                c4 -= c4[:baseline_end].mean()

                # ------------------------------------------------------------------
                # 4) Time‑domain features
                # ------------------------------------------------------------------
                row = {
                    "subject_id": sbj,
                    "session_id": sess,
                    "trial_id": tr,
                    # basic stats (C3)
                    "min_val_C3":   float(c3.min()),
                    "max_val_C3":   float(c3.max()),
                    "mean_val_C3":  float(c3.mean()),
                    "energy_C3":    float(np.sum(c3 ** 2)),
                    # basic stats (C4)
                    "min_val_C4":   float(c4.min()),
                    "max_val_C4":   float(c4.max()),
                    "mean_val_C4":  float(c4.mean()),
                    "energy_C4":    float(np.sum(c4 ** 2))
                }

                # ------------------------------------------------------------------
                # 5) ERD/ERS (8–30 Hz band power)
                # ------------------------------------------------------------------
                base_c3 = bandpower(c3[:baseline_end], 8, 30)
                task_c3 = bandpower(c3[baseline_end:], 8, 30)
                base_c4 = bandpower(c4[:baseline_end], 8, 30)
                task_c4 = bandpower(c4[baseline_end:], 8, 30)

                row.update({
                    "alpha_C3":  float(bandpower(c3, 8, 12)),
                    "alpha_C4":  float(bandpower(c4, 8, 12)),
                    "erd_C3_pct": 10 * np.log10(task_c3 / base_c3) if base_c3 else 0.0,
                    "erd_C4_pct": 10 * np.log10(task_c4 / base_c4) if base_c4 else 0.0
                })

                # ------------------------------------------------------------------
                # 6) Connectivity: coherence / imaginary coherence / PLV
                # ------------------------------------------------------------------
                f_coh, coh = signal.coherence(c3, c4, fs=fs, nperseg=fs * 2)
                mu_band = (f_coh >= 8) & (f_coh <= 30)
                coh_mu = float(coh[mu_band].mean()) if mu_band.any() else 0.0

                f_csd, Pxy = signal.csd(c3, c4, fs=fs, nperseg=fs * 2)
                f_pxx, Pxx = signal.welch(c3, fs=fs, nperseg=fs * 2)
                f_pyy, Pyy = signal.welch(c4, fs=fs, nperseg=fs * 2)
                imcoh = np.imag(Pxy) / np.sqrt(Pxx * Pyy)
                imcoh_mu = float(imcoh[(f_csd >= 8) & (f_csd <= 30)].mean())

                phi = np.angle(signal.hilbert(c3)) - np.angle(signal.hilbert(c4))
                plv_val = float(np.abs(np.exp(1j * phi).mean()))

                row.update({
                    "coh_C3C4_mu":   coh_mu,
                    "imcoh_C3C4_mu": imcoh_mu,
                    "plv_C3C4":      plv_val
                })

                # ------------------------------------------------------------------
                # 7) Higher‑order stats
                # ------------------------------------------------------------------
                row.update({
                    "range_C3": float(np.ptp(c3)),
                    "std_C3":   float(c3.std()),
                    "skew_C3":  float(skew(c3)),
                    "kurt_C3":  float(kurtosis(c3)),
                    "range_C4": float(np.ptp(c4)),
                    "std_C4":   float(c4.std()),
                    "skew_C4":  float(skew(c4)),
                    "kurt_C4":  float(kurtosis(c4))
                })

                # ------------------------------------------------------------------
                # 8) Frequency‑domain features (whole 0–40 Hz spectrum)
                # ------------------------------------------------------------------
                freqs = np.fft.rfftfreq(len(c3), 1 / fs)
                mask40 = freqs <= 40
                power_c3 = np.abs(np.fft.rfft(c3)) ** 2
                power_c4 = np.abs(np.fft.rfft(c4)) ** 2

                mean_f_c3 = np.sum(freqs * power_c3) / np.sum(power_c3)
                mean_f_c4 = np.sum(freqs * power_c4) / np.sum(power_c4)
                bw_c3 = np.sqrt(np.sum(((freqs - mean_f_c3) ** 2) * power_c3) / np.sum(power_c3))
                bw_c4 = np.sqrt(np.sum(((freqs - mean_f_c4) ** 2) * power_c4) / np.sum(power_c4))
                beta_c3 = np.sum(power_c3[(freqs >= 13) & (freqs <= 30)]) / len(c3)
                beta_c4 = np.sum(power_c4[(freqs >= 13) & (freqs <= 30)]) / len(c4)

                row.update({
                    "peak_freq_C3":   float(freqs[np.argmax(power_c3)]),
                    "peak_freq_C4":   float(freqs[np.argmax(power_c4)]),
                    "mean_freq_C3":   float(mean_f_c3),
                    "mean_freq_C4":   float(mean_f_c4),
                    "bw_C3":          float(bw_c3),
                    "bw_C4":          float(bw_c4),
                    "beta_power_C3":  float(beta_c3),
                    "beta_power_C4":  float(beta_c4),
                    "psd_mean_C3":    float(Pxx.mean()),
                    "psd_mean_C4":    float(Pyy.mean())
                })

                # ------------------------------------------------------------------
                # 9) Filter bank CSP‑like band powers
                # ------------------------------------------------------------------
                for lo, hi in fbcsp_bands:
                    mask = (freqs >= lo) & (freqs < hi)
                    row[f"fbcsp_{lo}_{hi}_C3"] = float(np.sum(power_c3[mask]) / len(c3))
                    row[f"fbcsp_{lo}_{hi}_C4"] = float(np.sum(power_c4[mask]) / len(c4))

                # Accumulate
                rows.append(row)
                X_all.append(np.stack([c3, cz, c4], axis=1))

    # ----------------------------------------------------------------------
    # 10) Common Spatial Patterns (CSP) on concatenated trials
    # ----------------------------------------------------------------------
    if X_all:
        X_all_arr = np.array(X_all)
        cov = np.mean([np.cov(trial.T) for trial in X_all_arr], axis=0)
        eigvals, eigvecs = np.linalg.eigh(cov)
        n_filt = min(4, eigvecs.shape[1])
        W = eigvecs[:, eigvals.argsort()[::-1][:n_filt]]

        for row, trial in zip(rows, X_all_arr):
            Z = trial @ W                         # spatially‑filtered signals
            var = np.var(Z, axis=0)
            logvar = np.log(var / var.sum())      # normalized log‑variance
            for k, v in enumerate(logvar, start=1):
                row[f"csp{k}"] = float(v)

    # ----------------------------------------------------------------------
    # 11) Save to CSV
    # ----------------------------------------------------------------------
    df = pd.DataFrame(rows)
    df.to_csv(output_csv, index=False)
    return df

make the preprocessing on the train , validation and test data

In [12]:
train_Data = preprocessing_and_loading(subjects_of_train,path_of_MI_diret_train,"MI_features_train.csv")
valid_Data = preprocessing_and_loading(subjects_of_valid,path_of_MI_diret_valid,"MI_features_valid.csv")
test_Data = preprocessing_and_loading(subjects_of_test,path_of_MI_diret_test,"MI_features_test.csv")

/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the ma

print all the features


In [13]:
print(train_Data.columns)

Index(['subject_id', 'session_id', 'trial_id', 'min_val_C3', 'max_val_C3',
       'mean_val_C3', 'energy_C3', 'min_val_C4', 'max_val_C4', 'mean_val_C4',
       'energy_C4', 'alpha_C3', 'alpha_C4', 'erd_C3_pct', 'erd_C4_pct',
       'coh_C3C4_mu', 'imcoh_C3C4_mu', 'plv_C3C4', 'range_C3', 'std_C3',
       'skew_C3', 'kurt_C3', 'range_C4', 'std_C4', 'skew_C4', 'kurt_C4',
       'peak_freq_C3', 'peak_freq_C4', 'mean_freq_C3', 'mean_freq_C4', 'bw_C3',
       'bw_C4', 'beta_power_C3', 'beta_power_C4', 'psd_mean_C3', 'psd_mean_C4',
       'fbcsp_8_12_C3', 'fbcsp_8_12_C4', 'fbcsp_12_16_C3', 'fbcsp_12_16_C4',
       'fbcsp_16_20_C3', 'fbcsp_16_20_C4', 'fbcsp_20_24_C3', 'fbcsp_20_24_C4',
       'fbcsp_24_28_C3', 'fbcsp_24_28_C4', 'fbcsp_28_32_C3', 'fbcsp_28_32_C4',
       'csp1', 'csp2', 'csp3'],
      dtype='object')


get the labels of the train and validation datasets

In [8]:
import pandas as pd

dataset_path

path_of_MI_diret_train_labels = os.path.join(dataset_path, "train.csv")
path_of_MI_diret_valid_labels = os.path.join(dataset_path, "validation.csv")

file_of_labels =  pd.read_csv(path_of_MI_diret_train_labels)
file_of_labels = file_of_labels[["subject_id","task","trial_session","trial","label"]]
file_of_labels = file_of_labels[file_of_labels["task"] == "MI"]
#print(file_of_labels.iloc[0:10,:])
sess1 = file_of_labels
#sess1 = sess1[file_of_labels["trial"]==1]
print(sess1)
file_of_labels_validation =  pd.read_csv(path_of_MI_diret_valid_labels)
file_of_labels_validation = file_of_labels_validation[["subject_id","task","trial_session","trial","label"]]
# Filter validation labels to include only 'MI' task
file_of_labels_validation = file_of_labels_validation[file_of_labels_validation["task"] == "MI"]
print(file_of_labels_validation)
sess2 = file_of_labels_validation

     subject_id task  trial_session  trial  label
0            S1   MI              1      1   Left
1            S1   MI              1      2  Right
2            S1   MI              1      3   Left
3            S1   MI              1      4   Left
4            S1   MI              1      5   Left
...         ...  ...            ...    ...    ...
2395        S30   MI              8      6   Left
2396        S30   MI              8      7  Right
2397        S30   MI              8      8  Right
2398        S30   MI              8      9  Right
2399        S30   MI              8     10   Left

[2400 rows x 5 columns]
   subject_id task  trial_session  trial  label
0         S31   MI              1      1   Left
1         S31   MI              1      2   Left
2         S31   MI              1      3  Right
3         S31   MI              1      4   Left
4         S31   MI              1      5   Left
5         S31   MI              1      6   Left
6         S31   MI              1      

label map 0=left , 1= right


In [15]:
# Map string labels to integers
label_map = {'Left': 0, 'Right': 1}
y_data = file_of_labels['label'].map(label_map).values  # shape: (2400,)
y_val_data = sess2['label'].map(label_map).values

# **The model training**

### the training is

*   loop on the combinations of the features to get the best validation accuracy
between all the combinations  
*   the combinations is between 15 to 30 feature

*   fit all the combination to 3 models (rf, xgb, svm) and take a soft voting between them







In [ ]:
from xgboost import XGBClassifier
from sklearn.svm import SVC
import joblib
from sklearn.ensemble import VotingClassifier
import numpy as np
import pandas as pd
import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from random import sample, seed

# Set seed for reproducibility
seed(42)
np.random.seed(42)

#full feature list
feature_names = ['min_val_C3', 'max_val_C3', 'mean_val_C3', 'energy_C3', 'min_val_C4', 'max_val_C4', 'mean_val_C4',
                 'energy_C4', 'alpha_C3', 'alpha_C4', 'erd_C3_pct', 'erd_C4_pct', 'coh_C3C4_mu', 'imcoh_C3C4_mu',
                 'plv_C3C4', 'range_C3', 'std_C3', 'skew_C3', 'kurt_C3', 'range_C4', 'std_C4', 'skew_C4', 'kurt_C4',
                 'peak_freq_C3', 'peak_freq_C4', 'mean_freq_C3', 'mean_freq_C4', 'bw_C3', 'bw_C4', 'beta_power_C3',
                 'beta_power_C4', 'psd_mean_C3', 'psd_mean_C4', 'fbcsp_8_12_C3', 'fbcsp_8_12_C4', 'fbcsp_12_16_C3',
                 'fbcsp_12_16_C4', 'fbcsp_16_20_C3', 'fbcsp_16_20_C4', 'fbcsp_20_24_C3', 'fbcsp_20_24_C4',
                 'fbcsp_24_28_C3', 'fbcsp_24_28_C4', 'fbcsp_28_32_C3', 'fbcsp_28_32_C4', 'csp1', 'csp2', 'csp3']
X_train_full =train_Data[feature_names]
y_train = y_data
X_val_full = valid_Data[feature_names]
y_val = y_val_data

# Standardize
scaler = StandardScaler()
X_train_full = scaler.fit_transform(X_train_full)
X_val_full = scaler.transform(X_val_full)

# Keep track of best
best_acc = 0
best_combo = None

# Try combinations of 15 to 30 features
for k in range(15, 31):
    tried_combos = set()
    for _ in range(100):  # Try 100 random combos per size
        combo = tuple(sorted(sample(range(len(feature_names)), k)))
        if combo in tried_combos:
            continue
        tried_combos.add(combo)

        selected_names = [feature_names[i] for i in combo]
        X_train = X_train_full[:, combo]
        X_val = X_val_full[:, combo]

        # Train
        ensemble = VotingClassifier(estimators=[
        ('rf', RandomForestClassifier(n_estimators=100)),
        ('xgb', XGBClassifier(n_estimators=100)),
        ('svm', SVC(kernel='linear', probability=True))
        ], voting='soft')

        ensemble.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = ensemble.predict(X_val)
        acc = accuracy_score(y_val, y_pred)

        if acc > best_acc:
            best_acc = acc
            best_combo = selected_names
            print(f"New best accuracy: {best_acc:.4f} with features: {best_combo}")
            # Save model and scaler
            joblib.dump(ensemble, "best_model.pkl")
            joblib.dump(scaler, "scaler_for_best_features2.pkl")
            joblib.dump(best_combo, "selected_feature_list2.pkl")
            print("Saved new best model, scaler, and feature list.")

print("\nFinal Best Accuracy and Features:")
print(f"Accuracy: {best_acc:.4f}")
print("Best Feature Combination:")
print(best_combo)

New best accuracy: 0.5000 with features: ['max_val_C3', 'mean_val_C3', 'max_val_C4', 'mean_val_C4', 'energy_C4', 'alpha_C3', 'plv_C3C4', 'range_C3', 'skew_C3', 'bw_C3', 'fbcsp_8_12_C4', 'fbcsp_16_20_C3', 'fbcsp_16_20_C4', 'fbcsp_20_24_C4', 'csp1']
Saved new best model, scaler, and feature list.
New best accuracy: 0.6000 with features: ['mean_val_C3', 'min_val_C4', 'max_val_C4', 'coh_C3C4_mu', 'plv_C3C4', 'kurt_C3', 'peak_freq_C3', 'peak_freq_C4', 'fbcsp_12_16_C3', 'fbcsp_12_16_C4', 'fbcsp_20_24_C3', 'fbcsp_20_24_C4', 'fbcsp_28_32_C3', 'fbcsp_28_32_C4', 'csp2']
Saved new best model, scaler, and feature list.
New best accuracy: 0.6400 with features: ['min_val_C4', 'mean_val_C4', 'erd_C3_pct', 'imcoh_C3C4_mu', 'range_C3', 'skew_C3', 'kurt_C4', 'peak_freq_C3', 'peak_freq_C4', 'beta_power_C3', 'fbcsp_8_12_C4', 'fbcsp_20_24_C4', 'fbcsp_24_28_C3', 'fbcsp_24_28_C4', 'csp1']
Saved new best model, scaler, and feature list.
New best accuracy: 0.6600 with features: ['mean_val_C3', 'min_val_C4', 'm